In [1]:
!pip freeze | grep scikit-learn

scikit-learn==1.5.0


In [2]:
!python -V

Python 3.10.13


In [ ]:
import pickle
import pandas as pd

In [2]:
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

/home/codespace/.local/lib/python3.12/site-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator DictVectorizer from version 1.5.0 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/codespace/.local/lib/python3.12/site-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator LinearRegression from version 1.5.0 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [3]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [6]:
def load_data(year, month):
    df = read_data(f'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_{year:04d}-{month:02d}.parquet')
    return df

### Q1. Notebook

In [7]:
# Run the codes for the March 2023 data.

year = 2023
month = 3

df = load_data(year, month)
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

In [ ]:
# What's the standard deviation of the predicted duration for this dataset?

y_pred.std()

np.float64(6.247488852238703)

### Q2. Preparing the output

In [14]:
# First, let's create an artificial ride_id column:
df_result = pd.DataFrame()
df_result['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')

In [15]:
# Next, write the ride id and the predictions to a dataframe with results.
df_result['prediction'] = y_pred

In [16]:
# Save it as parquet:
output_file = 'predictions.parquet'

df_result.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)

In [ ]:
# What's the size of the output file?

!ls -lh 'predictions.parquet'

-rw-rw-rw- 1 codespace codespace 66M Jun  8 04:04 predictions.parquet


### Q3. Creating the scoring script

In [ ]:
# Now let's turn the notebook into a script.

# Which command you need to execute for that?

!jupyter nbconvert --to script score.ipynb

### Q4. Virtual environment

In [ ]:
# Now let's put everything into a virtual environment.

!pip freeze | grep scikit-learn
!pipenv install scikit-learn==1.5.0 flask --python=3.10.13

In [ ]:
# The Pipfile.lock file keeps the hashes of the dependencies we use for the virtual env.
# What's the first hash for the Scikit-Learn dependency?

"sha256:057b991ac64b3e75c9c04b5f9395eaf19a6179244c089afdebaad98264bff37c"

### Q5. Parametrize the script

In [ ]:
# Let's now make the script configurable via CLI. We'll create two parameters: year and month.

# > please refer to the script file batch_predict.py

In [ ]:
# Run the script for April 2023.

!python batch_predict.py 2023 4

In [ ]:
# What's the mean predicted duration?

# > predicted mean duration: 14.292282936862449

### Q6. Docker container

In [ ]:
# please refer to the docker file homework.dockerfile

!docker build -t ride-duration-homework:v1 -f homework.dockerfile .

In [ ]:
# Now run the script with docker. What's the mean predicted duration for May 2023?

!docker run -it --rm -p 9696:9696 ride-duration-homework:v1 2023 5

# > predicted mean duration: 0.19174419265916945